In [51]:
%matplotlib qt
import numpy as np
from influence_moo.plotting import plot_grid, plot_mission, plot_pts, plot_vectors
from influence_moo.env.mission import Mission
from influence_moo.waves import sample_waves
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt

In [52]:
grid = np.ones((10,10),dtype=int)

grid[1,2] = 0
grid[4,1:5] = 0

grid[6,2] = 0
grid[9,1:5] = 0

plt.figure()
plot_grid(grid)

In [53]:
if False:
    np.savetxt("connectivity_grid.csv", grid, delimiter=",", fmt="%d")

In [54]:
if False:
    connect_df = pd.DataFrame(grid)
    connect_df.to_csv('connectivity.csv')

In [55]:
mission = Mission(Path("/home/gonzaeve/influence-multi-objective/results/captain_b/mission"))
plt.figure()
plot_mission(mission, include_waves=False)

In [56]:
# c
grid = np.ones((10,10),dtype=int)

grid[1,2] = 0
grid[4,:] = 0

grid[6,2] = 0
grid[9,1:5] = 0

plt.figure()
plot_grid(grid)

In [57]:
connect_df = pd.DataFrame(grid)
connect_df.to_csv('connectivity.csv')

In [58]:
""" Let's generate Captain d variants with different numbers of rows and columns
captain_d_r2_c2 (2 rows, 2 columns)
"""

n_rows = 10
n_cols = 4

obs_width = 5
gap_width = 3
x_dim = gap_width+(n_cols-1)*(obs_width+gap_width)

space = 5
y_dim = (n_rows-1)*space+space

grid = np.zeros((x_dim,y_dim), dtype=int)

for r in range(n_rows):
    grid[:, (r+1)*space:(r+1)*space+1] +=1

for c in range(n_cols):
    c_space = obs_width+gap_width
    grid[c_space*c+gap_width:c_space*(c+1),:] += 1

for r in range(grid.shape[0]):
    for c in range(grid.shape[1]):
        if grid[r,c] != 2:
            grid[r,c] = 0
        elif grid[r,c] == 2:
            grid[r,c] = 1

# Flip 1s and 0s
grid = 1 - grid

plt.figure()
plot_grid(grid)


In [59]:
"""Time to add in POIs"""

xs = [c*c_space+gap_width/2 for c in range(n_cols)]
ys = [(r+1)*space+1/2 for r in range(n_rows-1)]

In [60]:
"""Set up POIs with positions and values"""
poi_positions = []
poi_values = []

forward_order = True
base_val = 1.5

for xi, x in enumerate(xs):
    for yi, y in enumerate(ys):

        poi_positions.append(np.array([x,y]))

        # Value of POI depends on whether we are doing ascending y order or descending y order
        if forward_order:
            poi_values.append(base_val**(yi+1))
            # poi_values.append(yi+1)
        else:
            poi_values.append(base_val**(len(ys) - yi))
            # poi_values.append(len(ys) - yi)

    # Flip the order of ascending value for pois
    forward_order = not forward_order

"""Set up AUV paths"""
auv_paths = [[] for x in xs]
forward_order = True
for xi in range(len(auv_paths)):
    pois_in_path = poi_positions[xi*len(ys):(xi+1)*len(ys)]
    if not forward_order:
        pois_in_path = list(reversed(pois_in_path))

    if forward_order:
        start_pos = pois_in_path[0] - np.array([0, space*1/2])
    else:
        start_pos = pois_in_path[0] + np.array([0, space*1/2])
    auv_paths[xi].append(start_pos)

    for poi in pois_in_path:
        auv_paths[xi].append(poi)
    forward_order = not forward_order

"""Set up ASV start positions"""
asv_start_positions = []
asv_start_positions.append([grid.shape[0]/2, space/2])
for y in ys:
    asv_start_positions.append([grid.shape[0]/2,y+space/2])

"""Set up waves"""
def wave_x(x):
    a = 1.
    b = 10.
    c = 5.
    return a * np.sin(x/b + c)

def wave_y(y):
    a = 1.5
    b = .1
    c = 5.
    return a * np.sin(y/b + c)

In [61]:
plt.figure()
plot_grid(grid, cmap='tab10_r')
plot_pts(np.array(poi_positions), marker='o', fillstyle='full', linestyle='none',color='tab:green', markersize=10.0)

# Label each point
for i in range(len(poi_positions)):
    a = f"{float(poi_values[i]):.2f}"
    plt.annotate(a, (poi_positions[i][0], poi_positions[i][1]), textcoords="offset points", xytext=(0, 10), ha='center')

for p in auv_paths:
    plot_pts(np.array(p), color='purple')

for p in asv_start_positions:
    plot_pts(np.array([p]), marker='^', fillstyle='full', linestyle='none',color='orange')

wave_vectors = sample_waves(grid, 20, 20, wave_x, wave_y)
# print(grid.shape)
plot_vectors(wave_vectors, color='pink', lw=0.3)
plt.xlim([0, grid.shape[0]])
plt.ylim([0, grid.shape[1]])
plt.show()

In [50]:
connect_df = pd.DataFrame(grid)
connect_df.to_csv('connectivity.csv')

In [62]:
asv_start_pos_df = pd.DataFrame(asv_start_positions)
asv_start_pos_df.to_csv('asv_start_positions.csv')

In [63]:
for i, auv_path in enumerate(auv_paths):
    auv_path_df = pd.DataFrame(auv_path)
    auv_path_df.to_csv('path_'+str(i)+'.csv')

In [64]:
poi_pos_df = pd.DataFrame(poi_positions)
poi_pos_df.to_csv('pois.csv')

In [68]:
for p in poi_values:
    print(f"    - {{value: {p}, observation_radius: 3.0}}")

    - {value: 1.5, observation_radius: 3.0}
    - {value: 2.25, observation_radius: 3.0}
    - {value: 3.375, observation_radius: 3.0}
    - {value: 5.0625, observation_radius: 3.0}
    - {value: 7.59375, observation_radius: 3.0}
    - {value: 11.390625, observation_radius: 3.0}
    - {value: 17.0859375, observation_radius: 3.0}
    - {value: 25.62890625, observation_radius: 3.0}
    - {value: 38.443359375, observation_radius: 3.0}
    - {value: 38.443359375, observation_radius: 3.0}
    - {value: 25.62890625, observation_radius: 3.0}
    - {value: 17.0859375, observation_radius: 3.0}
    - {value: 11.390625, observation_radius: 3.0}
    - {value: 7.59375, observation_radius: 3.0}
    - {value: 5.0625, observation_radius: 3.0}
    - {value: 3.375, observation_radius: 3.0}
    - {value: 2.25, observation_radius: 3.0}
    - {value: 1.5, observation_radius: 3.0}
    - {value: 1.5, observation_radius: 3.0}
    - {value: 2.25, observation_radius: 3.0}
    - {value: 3.375, observation_radius